In [0]:
from google.colab import drive
drive.mount('/content/drive/')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=email%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdocs.test%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive.photos.readonly%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fpeopleapi.readonly&response_type=code

Enter your authorization code:
··········
Mounted at /content/drive/


In [0]:
cd /content/drive/My Drive/CS50_Final_Project/Video_Artistic_Style_Transfer

/content/drive/My Drive/CS50_Final_Project/Video_Artistic_Style_Transfer


In [0]:
# google colab does not come with torch installed

from os.path import exists
from wheel.pep425tags import get_abbr_impl, get_impl_ver, get_abi_tag
platform = '{}{}-{}'.format(get_abbr_impl(), get_impl_ver(), get_abi_tag())
cuda_output = !ldconfig -p|grep cudart.so|sed -e 's/.*\.\([0-9]*\)\.\([0-9]*\)$/cu\1\2/'
accelerator = cuda_output[0] if exists('/dev/nvidia0') else 'cpu'

!pip install -q http://download.pytorch.org/whl/{accelerator}/torch-0.4.1-{platform}-linux_x86_64.whl torchvision
import torch

tcmalloc: large alloc 1073750016 bytes == 0x58596000 @  0x7f8d2fbe82a4 0x591a07 0x5b5d56 0x502e9a 0x506859 0x502209 0x502f3d 0x506859 0x504c28 0x502540 0x502f3d 0x506859 0x504c28 0x502540 0x502f3d 0x506859 0x504c28 0x502540 0x502f3d 0x507641 0x502209 0x502f3d 0x506859 0x504c28 0x502540 0x502f3d 0x507641 0x504c28 0x502540 0x502f3d 0x507641


In [0]:
# we will verify that GPU is enabled for this notebook
# following should print: CUDA is available!  Training on GPU ...
# 
# if it prints otherwise, then you need to enable GPU: 
# from Menu > Runtime > Change Runtime Type > Hardware Accelerator > GPU

import numpy as np

# check if CUDA is available
train_on_gpu = torch.cuda.is_available()

if not train_on_gpu:
    print('CUDA is not available.  Training on CPU ...')
else:
    print('CUDA is available!  Training on GPU ...')

CUDA is available!  Training on GPU ...


In [0]:
# we need pillow version of 5.3.0
# we will uninstall the older version first
!pip uninstall -y Pillow
# install the new one
!pip install Pillow==5.3.0
# import the new one
import PIL
print(PIL.PILLOW_VERSION)
# this should print 5.3.0. If it doesn't, then restart your runtime:
# Menu > Runtime > Restart Runtime

Uninstalling Pillow-5.3.0:
  Successfully uninstalled Pillow-5.3.0
  Using cached https://files.pythonhosted.org/packages/62/94/5430ebaa83f91cc7a9f687ff5238e26164a779cca2ef9903232268b0a318/Pillow-5.3.0-cp36-cp36m-manylinux1_x86_64.whl
5.3.0


In [0]:
# Import the necessary packages:
import cv2
import glob
from PIL import Image							            # Python Image Library for image processing
import matplotlib.pyplot as plt 	            # Plotting
import numpy as np 								            # Numerical computation
import torch 								                  # Neural network computation
from torch import optim						            # optimizer to minimize the loss function
from torchvision import transforms, models		# Transformations on images and pre-trained models
import os, os.path 								            # To count the number of image files

In [0]:
# LOAD VGG19 (features only):
# vgg19.features 	: It consists of all the convolutional and pooling layers
# vgg19.classifier 	: It consists of the 3 linear classifier layers at the end

# We load in the pre-trained model and freeze the weights:
styleTransferModel = models.vgg19(pretrained=True).features

# Freeze all the VGG parameters since we are only optimizing the target image:
for params in styleTransferModel.parameters():
	params.requires_grad_(False)

Downloading: "https://download.pytorch.org/models/vgg19-dcbb9e9d.pth" to /root/.torch/models/vgg19-dcbb9e9d.pth
100%|██████████| 574673361/574673361 [00:06<00:00, 82195845.26it/s]


In [0]:
# Check if GPU is available:
if torch.cuda.is_available():
	device = torch.device("cuda")
	print("Running on GPU")
else:
	device = torch.device("cpu")
	print("Running on CPU")

# Move the model to the GPU if available:
styleTransferModel.to(device)

Running on GPU


Sequential(
  (0): Conv2d(3, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (1): ReLU(inplace)
  (2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (3): ReLU(inplace)
  (4): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  (5): Conv2d(64, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (6): ReLU(inplace)
  (7): Conv2d(128, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (8): ReLU(inplace)
  (9): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  (10): Conv2d(128, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (11): ReLU(inplace)
  (12): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (13): ReLU(inplace)
  (14): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (15): ReLU(inplace)
  (16): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (17): ReLU(inplace)
  (18): MaxPool2d(kernel_size=2, stride=2, padding=0, 

In [0]:
# Load the content and style images:
def load_image(image_path, max_size=400, shape=None):
	# Load in an image and make sure that it is <= 400 pixels in the X-Y dimension:
	# Convert the image to RGB:
	image = Image.open(image_path).convert('RGB')

	# Resize the image as a large image will slow down processing:
	if max(image.size) > max_size:
		img_size = max_size
	else:
		img_size = max(image.size)

	if shape is not None:
		img_size = shape

	img_transform = transforms.Compose([
										transforms.Resize(img_size),
										transforms.ToTensor(),
										transforms.Normalize((0.485, 0.456, 0.406),
															(0.229, 0.224, 0.225))])
	# Discard the transparent alpha channel (that's the :3) and add the batch dimension:
	image = img_transform(image)[:3, :, :].unsqueeze(0)

	return image

In [0]:
# CONTENT AND STYLE FEATURES:
# Map the layer names to the names given in the paper:
def get_features(image, model, layers=None):
	# Run an image forward through a model and get the features for a set of layers:

	# Layers for the content and style representation of an image:
	if layers is None:
		layers = {'0': 'conv1_1',
				  '5': 'conv2_1',
				  '10': 'conv3_1',
				  '19': 'conv4_1',
				  '21': 'conv4_2',		# Content representation
				  '28': 'conv5_1',}

	features = {}
	x = image
	# model._modules is a dictionary holding each module in the model:
	for name, layer in model._modules.items():
		x = layer(x)
		if name in layers:
			features[layers[name]] = x

	return features

In [0]:
# GRAM MATRIX:
# Define the gram matrix of the tensor:
def gram_matrix(tensor):
	# Calculate the Gram Matrix of a given tensor:
	# Get the batch_size, depth, height and width of the image:
	batch_size, depth, height, width = tensor.size()
	#(Sample tensor shape : torch.Size([1, 64, 400, 592]))
	# Here, batch_size = 1, depth = 64, height = 400, width = 592
	
	# Vectorize the input image tensor and add all the feature maps:
	tensor = tensor.view(depth, height * width)	
	# Transpose the image tensor:
	tensor_t = tensor.t()
	# Compute the gram matrix by multiplying the matrix by its transpose:
	gram = torch.mm(tensor, tensor_t)

	# Return the gram matrix:
	return gram

In [0]:
def convert_video_to_frames(video_directory):
	# Load the input video:
	capture_video = cv2.VideoCapture(video_directory)
	# Read the input frame:
	success, frame = capture_video.read()
	# Set counter for number of frames read:
	count = 1

	while success:
		# Save the frame that is read:
		cv2.imwrite('input_content_frames/frame_%d.jpg' % count, frame)
		# Read the next input frame:
		success, frame = capture_video.read()
		print("Read a new frame: ", count, "Success: ", success)
		count += 1

In [0]:
# Split the video into frames:
convert_video_to_frames('input_video/sample_video.mp4')

Read a new frame:  1 Success:  True
Read a new frame:  2 Success:  True
Read a new frame:  3 Success:  True
Read a new frame:  4 Success:  True
Read a new frame:  5 Success:  True
Read a new frame:  6 Success:  True
Read a new frame:  7 Success:  True
Read a new frame:  8 Success:  True
Read a new frame:  9 Success:  True
Read a new frame:  10 Success:  True
Read a new frame:  11 Success:  True
Read a new frame:  12 Success:  True
Read a new frame:  13 Success:  True
Read a new frame:  14 Success:  True
Read a new frame:  15 Success:  True
Read a new frame:  16 Success:  True
Read a new frame:  17 Success:  True
Read a new frame:  18 Success:  True
Read a new frame:  19 Success:  True
Read a new frame:  20 Success:  True
Read a new frame:  21 Success:  True
Read a new frame:  22 Success:  True
Read a new frame:  23 Success:  True
Read a new frame:  24 Success:  True
Read a new frame:  25 Success:  True
Read a new frame:  26 Success:  True
Read a new frame:  27 Success:  True
Read a new

In [0]:
# Function to un-normalize an image and convert from a Tensor image to a NumPy image for display or writing to disk:
def img_convert(tensor):
	# Display a tensor as an image:

	image = tensor.to("cpu").clone().detach()
	image = image.numpy().squeeze()
	image = image.transpose(1, 2, 0)
	image = image * np.array((0.229, 0.224, 0.225)) + np.array((0.485, 0.456, 0.406))
	image = image.clip(0, 1)

	return image

In [0]:
def apply_style_transfer(content_img_dir, style_img_dir):
	content_image_count = 1
	style_image_count = 1
	
	# Retrieve the total number of content image frames and style frames:
	num_content_imgs = len([name for name in os.listdir(content_img_dir) if os.path.isfile(os.path.join(content_img_dir, name))])
	num_style_imgs = len([name for name in os.listdir(style_img_dir) if os.path.isfile(os.path.join(style_img_dir, name))])
	# Divide the style images equally among the input frames:
	frames_with_current_style = num_content_imgs // (num_style_imgs - 1)

	# Since there are 249 frames in the test video, you can use for loop, but prefer using while to avoid using magic numbers (hardcoding)
	while(content_image_count <= num_content_imgs): 
	#for i in range(1, 250):
		# Load in the content and style images and move them to the GPU if available:
		content_image = load_image(content_img_dir + '/frame_' + str(content_image_count) + '.jpg').to(device)
		if(content_image_count % frames_with_current_style == 0):
			style_image_count += 1

		style_image = load_image(style_img_dir + '/style_' + str(style_image_count) + '.jpg', shape=content_image.shape[-2:]).to(device)

		# Retrieve the features:
		content_features = get_features(content_image, styleTransferModel)
		style_features = get_features(style_image, styleTransferModel)

		# Calculate the gram matrix for each of our style representations:
		style_grams = {layer: gram_matrix(style_features[layer]) for layer in style_features}

		# We create a 'target image'. Note that, we are starting with the content image and cloning it instead of creating an image with white filter:
		# We want to update our image based on the total loss and so we will turn on the gradients:
		target_image = content_image.clone().requires_grad_(True).to(device)

		# LOSS AND WEIGHTS:
		# We assign weights for each style layer. Weighting earlier layers more will result in *larger* style artifacts:
		# Notice we are excluding `conv4_2`, i.e. our content representation:
		style_weights = {'conv1_1': 1.,			# More style will come from earlier layers as they are weighted more
		                 'conv2_1': 0.8,
		                 'conv3_1': 0.2,
		                 'conv4_1': 0.2,
		                 'conv5_1': 0.2}		# Less style from later layers

		content_weight = 1  # alpha
		style_weight = 1e8  # beta, 1e6 = 1000000.0

		# Iteration hyperparameters:
		# Update the target image (as we update the model.parameters() in the classifiers):
		optimizer = optim.Adam([target_image], lr=0.003)
		# Number of iterations to update your image:
		steps = 2000
		show_every = 1000
		for ii in range(1, steps+1):

			# Get the features from the target image:
			target_features = get_features(target_image, styleTransferModel)
			
			# 1. Calculate the content loss:
			content_loss = torch.mean((target_features['conv4_2'] - content_features['conv4_2']) ** 2)

			# 2. Calculate the style loss:
			style_loss = 0
			# iterate through each style layer and add to the style loss:
			for layer in style_weights:
				# Get the "target" style representation for the layer:
				target_feature = target_features[layer]
				batch_size, depth, height, width = target_feature.shape

				# Calculate the target gram matrix:
				target_gram = gram_matrix(target_feature)

				# Get the "style" from the style gram matrices computed earlier:
				style_gram = style_grams[layer]

				# the style loss for one layer, weighted appropriately:
				layer_style_loss = style_weights[layer] * torch.mean((target_gram - style_gram) ** 2)

				# Add to the style loss:
				style_loss += layer_style_loss / (depth * height * width)

			# Calculate the total loss:
			total_loss = content_weight * content_loss + style_weight * style_loss

			# Update the target image:
			optimizer.zero_grad()			# zero out the gradients from previous iterations
			total_loss.backward()			# Backpropagate the loss
			optimizer.step()				# Update the target image

			# Display the intermediate results if required:
			#if ii % show_every == 0:
				#print("Total loss: ", total_loss.item())
				#plt.imshow(img_convert(target_image))
				#plt.show()

		# Save the style transferred image:
		target_image = img_convert(target_image)
		plt.imsave('output_style_transferred_frames/st_frame_%d.jpg' % content_image_count, target_image)
		print("completed style transfer on image: ", content_image_count)
		content_image_count += 1
		if(content_image_count <= 250):
			continue

In [0]:
# Apply style transfer on the input frames:
apply_style_transfer('input_content_frames', 'input_style_frames')

In [0]:
# Convert the final images to video:
def img_to_video(st_output_dir):
	img = cv2.imread(st_output_dir + '/st_frame_1.jpg')
	height, width, layers = img.shape
	fourcc = cv2.VideoWriter_fourcc(*'XVID')
	video = cv2.VideoWriter('output_processed_video/style_transfered_video.avi', fourcc, 25, (width, height))
	for i in range(1, 250):
		video.write(cv2.imread(st_output_dir + '/st_frame_' + str(i) + '.jpg'))

	cv2.destroyAllWindows()
	video.release()

img_to_video('output_style_transferred_frames')